Pytorch provide an API `torch.autograd` for automatic differentiation. This notebook contains mainly 7 parts:
1. `torch.autograd.function`
2. `torch.autograd.functional`
3. `torch.autograd.gradcheck`
4. `torch.autograd.anomaly_node`
5. `torch.autograd.grad_mode`
6. `model.eval()` and `torch.no_grad()`
6. `torch.autograd.profiler`

In [1]:
import torch

In [2]:
from torch.autograd.function import Function

In [24]:
class Exp(Function):                    # 此层计算e^x

    @staticmethod
    def forward(ctx, i):                # 模型前向
        print("forward begins")
        result = i.exp()
        ctx.save_for_backward(result)   # 保存所需内容，以备backward时使用，所需的结果会被保存在saved_tensors元组中；此处仅能保存tensor类型变量，若其余类型变量（Int等），可直接赋予ctx作为成员变量，也可以达到保存效果
        print(ctx)
        return result

    @staticmethod
    def backward(ctx, grad_output):     # 模型梯度反传
        print("backward begins")
        result, = ctx.saved_tensors     # 取出forward中保存的result
        print(ctx, grad_output)
        return grad_output * result     # 计算梯度并返回

In [25]:
x = torch.tensor([1.], requires_grad = True)
y = 2*x + 1
ret = Exp.apply(y)

forward begins


In [26]:
print(x.data, x.requires_grad, x.grad, x.grad_fn, x.is_leaf)

tensor([1.]) True None None True


In [27]:
print(y.data, y.requires_grad, y.grad, y.grad_fn, y.is_leaf)

tensor([3.]) True None <AddBackward0 object at 0x7fc52e396190> False


In [28]:
print(ret.data, ret.requires_grad, ret.grad, ret.grad_fn, ret.is_leaf)
print(ret.backward)

tensor([20.0855]) True None <torch.autograd.function.ExpBackward object at 0x7fc52e322910> False
<bound method Tensor.backward of tensor([20.0855], grad_fn=<ExpBackward>)>


In [29]:
ret.backward()

backward begins
<torch.autograd.function.ExpBackward object at 0x7fc52e322910> tensor([1.])


tensor([20.0855], grad_fn=<ExpBackward>)


In [23]:
print(x.grad, y.grad, ret.grad)

tensor([40.1711]) None None


None
